# Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę*: $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{|k|}&\text{ dla }k<0
\end{array}\right.,$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$ *logarytmem dyskretnym* $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle taka liczba istnieje.

Specyficzną sytuacją w teorii liczb jest, gdy $n=p$ jest liczbą pierwszą, a $q$ jest pierwiastkiem pierwotnym $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest **generatorem** grupy multiplikatywnej rzędu $p-1$,
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$.

## Algorytm baby-step giant-step

Jest to jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$.

### Krok 1. algorytmu:
- $m=\lceil\sqrt{p-1}\rceil$,
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$,
- obliczamy $r=(q^{-1})^m$.

### Krok 2. algorytmu:
- $b=a$,
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$,
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm,
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę.

## Zadanie 1.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1, 3, 2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1, 8, 6, 19, 7, 27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1, 76, 13, 84, 56, 75, 50, 71, 85, 19, 88]
r = 70
k = 3 (j = 0, i = 3)
```

In [1]:
# lekko zmodyfikowana klasa z poprzedniego laboratorium
class Zn:
    def __init__(self, val, N):
        self.val = val % N
        self.N = N

    def check_argument(self, argument):
        if not isinstance(argument, Zn):
            if isinstance(argument, int):
                return Zn(argument, self.N)
            raise TypeError(f"Argument operacji powinien być obiektem klasy Zn lub liczbą całkowitą! Otrzymano typ: {type(argument)}")
        elif argument.N != self.N:
            raise ValueError(f"Różne podstawy modularne N w argumentach: {self.N} oraz {argument.N}")

        return argument

    def is_negative_power(self, argument):
        return isinstance(argument, int) and argument < 0

    def find_modular_inverse(self):
        for i in range(1, self.N):
            if i * self.val % self.N == 1:
                return i

        raise ZeroDivisionError(f"{self.val} nie ma odwrotności modularnej dla podstawy modularnej {self.N}")
        
    def __eq__(self, other):
        return isinstance(other, Zn) and self.val == other.val and self.N == other.N
    
    def __add__(self, other):
        other = self.check_argument(other)
        return Zn(self.val + other.val, self.N)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        other = self.check_argument(other)
        return Zn(self.val - other.val, self.N)

    def __rsub__(self, other):
        other = self.check_argument(other)
        return other - self

    def __mul__(self, other):
        other = self.check_argument(other)
        return Zn(self.val * other.val, self.N) 

    def __rmul__(self, other):
        return self * other

    def __pow__(self, other):
        if self.is_negative_power(other):
            power = -other
            mod_inv = self.find_modular_inverse()
            return Zn(mod_inv ** power, self.N)
        else:
            other = self.check_argument(other)
            return Zn(self.val ** other.val, self.N)

    def __rpow__(self, other):
        other = self.check_argument(other)
        return other ** self

    def __str__(self):
        return self.__repr__()
    
    def __repr__(self):
        return str(self.val)

In [2]:
from math import ceil

def baby_step(p, q):
    q = Zn(q, p)
    m = ceil((p - 1) ** 0.5)
    powers = {}
    
    for i in range(m):
        powers[i] = (q ** i)
    
    r = q ** -m
    
    return m, powers, r

In [3]:
def giant_step(p, a, m, powers, r):
    a = Zn(a, p)
    b = a
    
    for j in range(m):
        if b in powers.values():
            i = list(powers.values()).index(b)
            return j * m + i
        
        b *= r

Przetestujmy działanie algorytmu dla podanych przykładów.

In [4]:
# TEST 1
p = 7
q = 3
a = 4

m, powers, r = baby_step(p, q)
k = giant_step(p, a, m, powers, r)

# m = 3, tablica_testowa = [1, 3, 2], r = 6, k = 4
print(f"m = {m}, tablica_testowa = {list(powers.values())}, r = {r}, k = {k}")

m = 3, tablica_testowa = [1, 3, 2], r = 6, k = 4


In [5]:
# TEST 2
p = 29
q = 8
a = 10

m, powers, r = baby_step(p, q)
k = giant_step(p, a, m, powers, r)

# m = 6, tablica_testowa = [1, 8, 6, 19, 7, 27], r = 9, k = 17
print(f"m = {m}, tablica_testowa = {list(powers.values())}, r = {r}, k = {k}")

m = 6, tablica_testowa = [1, 8, 6, 19, 7, 27], r = 9, k = 17


In [6]:
# TEST 3
p = 113
q = 76
a = 84

m, powers, r = baby_step(p, q)
k = giant_step(p, a, m, powers, r)

# m = 11, tablica_testowa = [1, 76, 13, 84, 56, 75, 50, 71, 85, 19, 88], r = 70, k = 3
print(f"m = {m}, tablica_testowa = {list(powers.values())}, r = {r}, k = {k}")

m = 11, tablica_testowa = [1, 76, 13, 84, 56, 75, 50, 71, 85, 19, 88], r = 70, k = 3


# Wymiana klucza typu Diffie-Hellman

Alice i Bob uzgadniają klucz publiczny, będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$,
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$,
- Alice generuje $x=q^n\mod p$ i wysyła do Boba,
- Bob generuje $y=q^m\mod p$ i wysyła Alice,
- Alice oblicza klucz $k=y^n\mod p$,
- Bob oblicza klucz $k=x^m\mod p$.


## Zadanie 2.

Zaimplementuj powyższy algorytm wymiany klucza. Dobierz parametry $p$ i $q$ tak, żeby znając $x$, $y$, $p$ i $q$ nie dało się odtworzyć sekretów algorytmem z zadania 1.

In [7]:
def dh_key_exchange(p, q, n, m):
    x = pow(q, n, p)
    y = pow(q, m, p)
    
    kA = pow(y, n, p)
    kB = pow(x, m, p)
    
    return kA, kB, x, y

Sekrety Alice i Boba powinny być **losowane** ze zbioru $\{1, 2, \dots p-1\}$.

In [8]:
from random import randint

Sprawdźmy, czy algorytm działa prawidłowo, tzn. czy klucze są takie same oraz czy na podstawie $x$ i $y$ (oraz liczb $p$ i $q$) zostaną poprawnie odczytane *sekrety*.

In [9]:
# TEST 1
p = 29
q = 8
n = randint(1, p) # sekret Alice
m = randint(1, p) # sekret Boba

# chcemy później porównać "złamane" wartości
print(f"Sekret Alice = {n}, sekret Boba = {m}")

kA, kB, x, y = dh_key_exchange(p, q, n, m)
print(f"Klucz Alice = {kA}, klucz Boba = {kB}, x = {x}, y = {y}")

assert kA == kB
print("Klucze się zgadzają!")

Sekret Alice = 3, sekret Boba = 20
Klucz Alice = 7, klucz Boba = 7, x = 19, y = 16
Klucze się zgadzają!


Sprawdźmy, czy dla tych danych uda się odzyskać tajne wartości $n$ i $m$.

In [10]:
# funkcja bazująca na rozwiązaniu zadania 1., obliczająca algorytm dyskretny
def discrete_logarithm(p, q, a):
    m, powers, r = baby_step(p, q)
    k = giant_step(p, a, m, powers, r)
    return k

In [11]:
# próba "złamania" algorytmu, tzn. odczytania sekretów
print(f"Złamany sekret Alice = {discrete_logarithm(p, q, x)}")
print(f"Złamany sekret Boba = {discrete_logarithm(p, q, y)}")

Złamany sekret Alice = 3
Złamany sekret Boba = 20


Jak widać, sekrety są łamane i to bardzo szybko (bo sprawdziliśmy małe liczby). Spróbujmy zatem dla większych wartości $p$ i $q$. Uznajemy, że jeśli czas obliczeń przekroczy 10 minut, to algorytm nie został złamany.

---

Do wygenerowania liczb pierwszych korzystam z [tej strony](https://bigprimes.org/).

Z kolei do sprawdzenia pierwiastków pierwotnych korzystam z [widgetu Wolfram Alpha](https://www.wolframalpha.com/widgets/view.jsp?id=ef51422db7db201ebc03c8800f41ba99), który umożliwia takie obliczenia.

In [12]:
# TEST 2
p = 6226665521 # 10-cyfrowa liczba pierwsza
q = 3 # pierwiastek pierwotny (najmniejszy)
n = randint(1, p)
m = randint(1, p)

print(f"Sekret Alice = {n}, sekret Boba = {m}")

kA, kB, x, y = dh_key_exchange(p, q, n, m)
print(f"Klucz Alice = {kA}, klucz Boba = {kB}, x = {x}, y = {y}")

assert kA == kB
print("Klucze się zgadzają!")

print(f"Złamany sekret Alice = {discrete_logarithm(p, q, x)}")
print(f"Złamany sekret Boba = {discrete_logarithm(p, q, y)}")

Sekret Alice = 720172183, sekret Boba = 336206692
Klucz Alice = 2431131538, klucz Boba = 2431131538, x = 1927140212, y = 3863344769
Klucze się zgadzają!
Złamany sekret Alice = 720172183
Złamany sekret Boba = 336206692


Dla 10-cyfrowej liczby pierwszej złamanie klucza zajęło około 10 minut (ale na jeden klucz - łącznie 20 minut). Wydaje się to dość długo, możliwe że wynika ze sposobu implementacji algorytmu z zadania 1.

Aby się upewnić, sprawdźmy jeszcze działanie dla 20-cyfrowej liczby pierwszej.

In [ ]:
# TEST 3
p = 92241790942553929493 # 20-cyfrowa liczba pierwsza
q = 2 # najmniejszy pierwiastek pierwotny
n = randint(1, p)
m = randint(1, p)

print(f"Sekret Alice = {n}, sekret Boba = {m}")

kA, kB, x, y = dh_key_exchange(p, q, n, m)
print(f"Klucz Alice = {kA}, klucz Boba = {kB}, x = {x}, y = {y}")

assert kA == kB
print("Klucze się zgadzają!")

print(f"Złamany sekret Alice = {discrete_logarithm(p, q, x)}")
print(f"Złamany sekret Boba = {discrete_logarithm(p, q, y)}")

W tym przypadku po odczekaniu 10 minut wciąż obliczenia się nie zakończyły (nawet dla sekretu Alice), dlatego je przerwałem. Można uznać, że algorytm nie został złamany.